<a href="https://colab.research.google.com/github/jfernandoghe/Evaluating-eye-tracker-calibration-models-with-the-Akaike-information-criterion/blob/master/Evaluating_eye_tracker_calibration_models_with_the_Akaike_information_criterion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Download dataset
Options: Background luminance was varied systematically from black to white in 7 steps (0, 12.5, 25, 37, 50, 75 and
100% of the available screen luminance). Individual information per eye, 3 repetitions per user.

In [1]:
!mkdir PreProcessed
!mkdir OutputTechniques
!mkdir OutputMetrics
!apt-get install git-lfs
!cd PreProcessed && git lfs clone https://github.com/jfernandoghe/Dataset.git

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  git-lfs
0 upgraded, 1 newly installed, 0 to remove and 6 not upgraded.
Need to get 2,129 kB of archives.
After this operation, 7,662 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 git-lfs amd64 2.3.4-1 [2,129 kB]
Fetched 2,129 kB in 1s (2,228 kB/s)
Selecting previously unselected package git-lfs.
(Reading database ... 131304 files and directories currently installed.)
Preparing to unpack .../git-lfs_2.3.4-1_amd64.deb ...
Unpacking git-lfs (2.3.4-1) ...
Setting up git-lfs (2.3.4-1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
          with new flags from 'git clone'

'git clone' has been updated in upstream Git to have comparable
speeds to 'git

## Obtain latest modified RANSAC Regressor from public repository
Modified version of original sklearn RANSAC Regressor

In [2]:
try:
  !rm -r /content/RANSACRegressor2
  !git clone https://github.com/jfernandoghe/RANSACRegressor2.git   
except:
  !git clone https://github.com/jfernandoghe/RANSACRegressor2.git        

rm: cannot remove '/content/RANSACRegressor2': No such file or directory
Cloning into 'RANSACRegressor2'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 18 (delta 4), reused 8 (delta 1), pack-reused 0
Unpacking objects: 100% (18/18), done.


## Define and import modules, classes, functions and packages



In [0]:

#@title
import sys
import pandas as pd
import numpy as np
import numpy.linalg as lalg
from numpy import where
import io
import re
import random
from itertools import combinations, chain
import math
from math import log
import itertools
import csv
import statistics
import scipy

def powerset(input_set, terms):
    print('terms ', terms)
    result = []
    for size in range(terms + 1):
        result += combinations(input_set, size)
    return result
      
class TipoPrueba:
  def __init__(self, pant, med, grouping): 
    self.pant = pant
    self.med = med
    self.grouping = grouping

  def __repr__(self):
    return "Ajustar columnas de mediciones %s a las de pantalla %i con agrup %i" \
  % (self.med, self.pant, self.grouping)

def modeval_ransac(A,b):
  parnum = A.ndim
  numite = 100
  AIC = 0
  AICc = 0
  MSE = 0
  return AIC, AICc, MSE

def c_pow(val, n):
  return [ val**j  for j in range(0,n+1)]
          
          
def allTerms(valXY, m, n):
  v_m = c_pow(valXY[0], m)
  v_n = c_pow(valXY[1], n)
  mmax = max(m,n)
  vals = [v_m[a]*v_n[b] for a in range(m+1) for b in range(n+1) if a+b<= mmax]
  return vals

def allSymbTerms(m, n):
  sym_m = [ f'x^{j}' if j>1 else ('x' if j == 1 else '')  for j in range(0,m+1)]
  sym_n = [ f'y^{j}' if j>1 else ('y' if j == 1 else '')  for j in range(0,n+1)]
  mmax = max(m,n)
  symb = [sym_m[a]+sym_n[b] if len(sym_m[a]+sym_n[b])>0 else '1' for a in range(m+1) for b in range(n+1) if a+b<= mmax] 
  return symb

################
#  Convert pixels to angle (in degrees)
#     parameters: 
#     v1, pixel position
#     smm, monitor size (mm)
#     d,   eye distance
def dst2Degree(v1, d, smm, spx):
  # distance to screen center (in pixels)
  v1c = v1 - spx/2 
  
  # convert pixel position in pixels to mm
  v1mm = (v1c*smm)/spx
  
  
  # calculate and return angle b 
  #      v1mm
  #     ____ 
  #     |  /
  #  d  |b/
  #     |/ 
  return math.degrees( math.atan2(v1mm, d) )



def removeSpurious(a, perc):
  num = int(round(len(a) * perc/100.0))
  for i in range(num):
    a = np.delete(a, a.argmax()) 
  return a


def removeSpuriousN(a, num):
  for i in range(num):    
    a = np.delete(a, a.argmax()) 
  return a

  

## Main algorithm

In [5]:
from google.colab import files
import glob
from RANSACRegressor2 import RANSACRegressor2
# Local Variables
max_m = 3
max_n = 2
mn = (max_m+1)*(max_n+1)
max_degreeterm = 2
t = TipoPrueba(1, (3,4), 5) # X configuration 
# t = TipoPrueba(2, (3,4), 5) # Y configuration 
percData = 8
coor = [ {228,370,512,654,796,100,242,384,526,668}, {470,655,840,1025,1210,155,340,525,710,895} ] #Coordinates of calibrations points
res = [ [1600,1200], [1920, 1080] ] #Resolution in pixeles
Smm = [ [487, 274], [508, 406.4] ] #Width and Height in mm


path = "/content/PreProcessed/Dataset/Data/*.csv" 
for fname in glob.glob(path):
  df = pd.read_csv(fname, sep=';', header=None, names=["Participant", "Xp", "Yp", "x", "y", "CalPoint", "tStamp", "d_eye"])

  # CSV Read
  # Header: Participant, x', y', x, y, CalPoint :
  #df = pd.read_csv('/content/PupilDynamics_Right.csv', sep=';', header=None, names=["Participant", "Xp", "Yp", "x", "y", "CalPoint", "tStamp", "dRight", "dLeft"])
  #df = pd.read_csv('/content/drive/My Drive/06_Public/03_Datasets/G_Right.csv', sep=';', header=None, names=["Participant", "Xp", "Yp", "x", "y", "CalPoint", "tStamp", "dLeft", "dRight"])
#   df = pd.read_csv('/content/PreProcessed/PupilDynamics_Right_l1.csv', sep=';', header=None, names=["Participant", "Xp", "Yp", "x", "y", "CalPoint", "tStamp", "d_eye"])
  df = df[df.x != 0]
  df = df[df.y != 0]

  df = df[df.Xp != 0]
  df = df[df.Yp != 0]
  # df = df[df.dLeft != 0]
  # df = df[df.dRight != 0]
  df = df[df.d_eye != 0]
  df = df.fillna(0)
  df = df.values
  M = np.array(df)



  #subsampling
  M = M[::20, : ]



  users = np.unique(M[:,0])
  # users = [5]
  #print(users)
  #users = np.concatenate( [users[:14],users[16:]] )


  #print("Terms --->", allTermStr)
  print('___________________________________________')
  for part in users:
    print("")
    print("************************************************************************************")
    print ('Participant ', part)
    print("************************************************************************************")
    print("")

    mPart = M[where(M[:, 0] == part)] # Participant' data
    bb = set(mPart[:,1])     #Ejemplo de X, índice [0] 
    
    
    
#***************************************
    if bool(bb & coor[0]):   #Check screentype
      mon = 0
    elif bool(bb & coor[1]):
      mon = 1    
#***************************************
    
    
    b = mPart[:, t.pant]
    d = mPart[:, t.med]   # XY data

    group = mPart[:, t.grouping] 
    segmList = [where(group==i)[0] for i in range(1,26)]
    allTermStr = allSymbTerms (max_m, max_n)
    s1 = np.asarray( [allTerms(xy, max_m, max_n) for xy in d] )


    setA = {1, 17, 24, 13, 10, 19, 7, 21, 3} #Training set
    setB = {i for i in range (25) } #Total set
    setC = setB - setA #Validation set

    if ( len(set( np.unique(mPart[:,5]).flatten().astype(int)-1 ).symmetric_difference(setB))==0 ):


      segmLi1 = [segmList[i] for i in setA] # training-measurements
      merged1 = list(itertools.chain.from_iterable(segmLi1))
      segmLi2 = [segmList[i] for i in setC]   # test-measurements



      # common approach
      # (a) remove the first 10 percent of measurements (saccadic jumps SJ)
      segmLstNSJ = [ lst[math.ceil(len(lst)/percData):] for lst in segmLi1] # list without saccadic_jumps
      # (b) pick the median value
      b_mm = [ statistics.median_high(b[lst]) for lst in segmLstNSJ]
      d_mm = [ [statistics.median_high(d[lst,0]),statistics.median_high(d[lst,1])] for lst in segmLstNSJ] # XY data
      s1_mm = np.asarray( [allTerms(xy, max_m, max_n) for xy in d_mm] )


      #models =  powerset([i for i in range(s1.shape[1])], len(segmLi1))[1:]

      #Terms   0    1     2     3    4       5      6      7       8
      #Terms ['1', 'y', 'y^2', 'x', 'xy', 'xy^2', 'x^2', 'x^2y', 'x^3']
      #models =[[3,0]]
      #models = [[3,0], [4,0], [5,0], [6,0], [7,0], [8,0] , [4, 3, 0]]
      
      # X-Models  0        1          2          3          4          5            6             7            8             9             10            11             12
      models = [[3,0], [3, 1, 0], [4, 3, 0], [6, 3, 0], [8, 3, 0], [6, 3, 0], [4, 3, 1, 0], [8, 6, 3, 0], [1, 6, 3, 0], [4, 6, 3, 0], [1, 8, 3, 0], [2, 8, 3, 0], [4, 8, 3, 0] ]

      # Y-Models
      # models = [[1, 0], [3, 1, 0], [4, 1, 0], [2, 1, 0], [1, 6, 0], [1, 4, 3, 0],[2, 1, 3, 0], [5, 3, 1, 0], [4, 1, 6, 0], [7, 6, 1, 0]] 


      selStr = ["AIC", "AICc", "KIC", "KICc", "AICF", "KICc", "RAIC"] 


      # selectors explored: AIC, AICc, AKIC, AKICc, AICF, KICc, RAIC
      errR2_4all = np.zeros((len(models), len(setA)))

#       rThreshold = 40
      alp = 3
      de = 60
      rThreshold = (res[mon][t.grouping-5] *  (de*math.degrees(math.atan(alp))))/Smm[mon][t.grouping-5]
  
      modelNum =0
      errorsEv = np.zeros([len(setC)])
      for model in models: 
          maxBits = len(allTermStr)
          if len(model)>0: # and len(model)<=max_degreeterm: # Select Models that accomplish certain parameter conditions (e.g., up to 5 parameters)
            A = s1[:,model]


            ######### training phase
            ########################
            allOk = True
            try:
              #  Fit Simple Linear model (1)
              Am = A[merged1, :]
              bm = b[merged1] 
              soln =  lalg.lstsq(Am, bm, rcond=None)
              theta = soln[0]
              residues = soln[1]


              # Fit linear model with point selection (median)  (2)
              A_mm = s1_mm[:,model]
              soln_mm =  lalg.lstsq(A_mm, b_mm, rcond=None)
              theta_mm = soln_mm[0]
              residues_mm = soln_mm[1]


              # Robustly fit linear model with RANSAC algorithm (Robust Linear Model) (3)
              ransac = RANSACRegressor2.RANSACRegressor2(residual_threshold=rThreshold)
              ransac.fitSegm(A, b, segmLi1)

              pos = 0
              for li in segmLi1:
                Avalid = np.array( [A[i, :] for i in li] )
                bvalid = np.array( [b[i] for i in li] )

                errR = ransac.predict(Avalid)-bvalid
                errR2 = np.multiply(errR, errR)
                errR2_4all[modelNum][pos] =  np.percentile(errR2, 50)
                pos += 1


            except ValueError as e:
              allOk = False
              print('errX', e)




            #end_try  




            ######### evaluation phase
            ########################
            pos1 = 0
            betaR = np.zeros([len(setC)])
            betaS = np.zeros([len(setC)])
            betaC = np.zeros([len(setC)])


            # Calculate and save errors

            for li in segmLi2:
              Atest = np.array( [A[i, :] for i in li] )
              btest = np.array( [b[i] for i in li] )
              disL = np.array( [(mPart[i, 7])/10 for i in li] ) 

              # estimation from a simple model (1)
              estimated =  np.matmul(Atest,theta)

              # estimation from saccadic_and_median filter (2)
              estimated_mm =  np.matmul(Atest,theta_mm)

              # estimation from Ransac (2)
              if allOk:    
                estimated_rr = ransac.predict(Atest) 
              else:
                estimated_rr = 9999              

#               # error in DEGREES
#               if bool(bb & coor[0]):
#                 mon = 0
#               elif bool(bb & coor[1]):
#                 mon = 1


              ang  = [dst2Degree(btest[b], disL[b], Smm[mon][t.grouping-5], res[mon][t.grouping-5]) for b in range(len(disL))]
              ang = np.array(ang)
              ranAng = [dst2Degree(estimated_rr[b], disL[b], Smm[mon][t.grouping-5], res[mon][t.grouping-5]) for b in range(len(disL))]
              comAng = [dst2Degree(estimated[b],    disL[b], Smm[mon][t.grouping-5], res[mon][t.grouping-5]) for b in range(len(disL))]
              simAng = [dst2Degree(estimated_mm[b], disL[b], Smm[mon][t.grouping-5], res[mon][t.grouping-5]) for b in range(len(disL))]

              betaR[pos1] = np.median(abs(np.array(ranAng)- ang))
              betaS[pos1] = np.median(abs(np.array(comAng)- ang))
              betaC[pos1] = np.median(abs(np.array(simAng)- ang))



              pos1 += 1

            #end_for



            usedTerms = [allTermStr[o] for o in model]

            mR = np.median(betaR)            
            mS = np.median(betaS)
            mC = np.median(betaC)



            print("modelNum", modelNum )
            print(usedTerms)

            print(f'Ransac:  {mR}')
            print("Simple: ", mS )
            print("Common: ", mC )
            print("-----")
            errorsEv[modelNum] = mR

            with open('/content/OutputTechniques/'+fname[-10:], 'a') as csvFile:
              writer = csv.writer(csvFile, delimiter=';')

              # participant, model, strategy, modelName, value
              writer.writerow([part, modelNum, 0, usedTerms, mR])
              writer.writerow([part, modelNum, 1, usedTerms, mS])
              writer.writerow([part, modelNum, 2, usedTerms, mC])
            csvFile.close()      
            modelNum += 1


      #print(errR2_4all)
      ################### simulate selection
      inAll = errR2_4all > rThreshold*rThreshold*1.2*1.2   
      toDelete = min(inAll.sum(axis = 1))
      #print("points to delete", toDelete)
      bestSelectorVal = np.ones(7) *  np.inf
      selectorError = np.zeros(7)    
      
      modelNum = 0
      for model in models: # evaluate each model      
        k = len(model)
        n = len(setA)-toDelete
        if 2*k <= n:
          rssI = removeSpuriousN(errR2_4all[modelNum], toDelete)
          #print("rssI", rssI)
          sumRsc = np.sum(rssI)
          c = rThreshold * rThreshold
          sigmaHat = sumRsc/n
          errRnorm =  rssI/sigmaHat
          rhoError = [(r*r)/2 if abs(r)<=c else c*r*r- c*c/2  for r in errRnorm]

          AIC = 2*(k) + n*math.log(sumRsc/n)
          AICc = AIC + (2*k**2 + 2*k )/ (n - k - 1) if n>k+1 else -999
          KIC = n*math.log(sumRsc/n) + 3*(k+1)
          KICc = n*math.log(sumRsc/n) + (2*(k+1)*n)/(n-k-2) - n*scipy.special.digamma((n-k)/2)*((n-k)/2) + n* math.log(n/2) if n>k+2 else -999 # definir PSI
          AKICc = n*math.log(sumRsc/n) + ((k+1)*(3*n-k-2))/(n-k-2) + k/(n-k)  if n>k and n>k+2 else -999
          AICF = n/math.log(sumRsc/n) * ((n*k)/(n-k-2) + 2) if n>k+2 else -999
          RAIC = 2*sum(rhoError) + 2*k
          
          selectorValues = [AIC, AICc, KIC, KICc, AICF, KICc, RAIC]
          for v in range (len(selectorValues)):
            if selectorValues[v] < bestSelectorVal[v]:
              bestSelectorVal[v] = selectorValues[v]
              selectorError[v] = errorsEv[modelNum]
          
        else:
          AIC, AICc, KIC, KICc, AKICc, AICF, RAIC = -999,-999,-999,-999,-999,-999,-999
                
        modelNum += 1
        
      #print("selectorError", selectorError)
      with open('/content/OutputMetrics/'+fname[-10:], 'a') as csvFile:
        writer = csv.writer(csvFile, delimiter=';')

        ss = 0  
        modelSe = np.where(selectorError[0]==errorsEv) if selectorError[0]!=0 else [-999]
        print("Model Selected", int(modelSe[0]))
        for mSel in selectorError:
                    # participant, model, strategy, modelName, value
          writer.writerow([part, modelSe[0], ss+1000, selStr[ss], mSel])  # participant,  selNum,  selName, value
          ss += 1
      csvFile.close()
    else:
      print("Not enough calibration points")
    print('___________________________________________')

___________________________________________

************************************************************************************
Participant  1.0
************************************************************************************

modelNum 0
['x', '1']
Ransac:  2.9978213669677354
Simple:  2.997821366967738
Common:  1.284346106384902
-----
modelNum 1
['x', 'y', '1']
Ransac:  2.6857536678239637
Simple:  2.685753667823088
Common:  1.0017798760206897
-----
modelNum 2
['xy', 'x', '1']
Ransac:  2.9578848524573624
Simple:  2.957884852457415
Common:  0.6748559972515318
-----
modelNum 3
['x^2', 'x', '1']
Ransac:  2.666961411650597
Simple:  2.6669614116505995
Common:  1.1459546095580477
-----
modelNum 4
['x^3', 'x', '1']
Ransac:  2.3584561975435663
Simple:  2.3584561975435485
Common:  1.1435019261070343
-----
modelNum 5
['x^2', 'x', '1']
Ransac:  2.666961411650597
Simple:  2.6669614116505995
Common:  1.1459546095580477
-----
modelNum 6
['xy', 'x', 'y', '1']
Ransac:  3.0007946112801136
Simple: 

In [0]:
!cd OutputMetrics    && cat *.csv > AllM.csv
!cd OutputTechniques && cat *.csv > AllT.csv